In [4]:
### PLEASE COPY NOTEBOOKS TO YOUR FOLDERS TO PREVENT COMMIT CONFLICTS
from dcicutils import ff_utils
from functions.notebook_functions import *
import json

# get key from keypairs.json
my_env = 'data'
my_key = get_key('koray_data')
schema_name = get_schema_names(my_key) 
print('WORKING ON', my_key['server'], '\n')

##### COLLECT ITEMS TO Release #####
# use either a starting item to fetch all linked items

# Use a starting item to find linked ones
# starting_items = ['46db06ad-b399-4cf4-9acc-07b3e25ef132']
#add_items = get_query_or_linked(my_key, linked=starting_items)

# or a search query
#my_query = '/search/?q=GOLD&type=Item&limit=all'
#add_items = get_query_or_linked(my_key, query=my_query)

# if you want you can dump them to separate json files (will work as test insert)
# dump_to_json(add_items, destination folder)

# my_query = '/search/?biosample.biosource.individual.organism.name=mouse&biosample.biosource_summary=ES-E14&experiment_type=in%20situ%20Hi-C&type=ExperimentHiC'
# store = get_query_or_linked(my_key, query=my_query)
# print(store.keys())
# print(len([i['uuid'] for key in store for i in store[key]]))
# print()

find_linked = ['37adea6e-6a27-41af-bf03-1f2edc8cbf91']
# store = get_query_or_linked(my_key, linked=find_linked, linked_frame='raw', ignore_field=['references', 'experiment_relation', 'biosample_relation'])

store, uuid = record_object_es(find_linked, my_key, schema_name, store_frame='raw', add_pc_wfr=False, ignore_field=['experiment_relation', 'biosample_relation', 'references'])
# print(store.keys())
# for key in store:
#     print(key, len(store[key]))
# print(len([i['uuid'] for key in store for i in store[key]]))

print()
for key in store:
    print(key, len(store[key]))
print(len([i['uuid'] for key in store for i in store[key]]))

WORKING ON https://data.4dnucleome.org 

found it in b410b0b7-5fad-4ceb-b3f1-80b673a4cb91 ExperimentHiC before removal

enzyme 1
treatment_agent 1
file_format 1
ontology_term 56
biosource 1
organism 1
user 9
quality_metric_fastqc 32
lab 6
award 5
experiment_set_replicate 1
ontology 3
experiment_hi_c 2
vendor 1
protocol 2
individual_mouse 1
file_fastq 32
biosample 2
157


In [5]:
#### This part should only run once!

transfer_env = 'fourfront-webdev'
transfer_key = ff_utils.get_authentication_with_server({}, ff_env=transfer_env)

#### This part should only run once!

# if the item exist in the target, should it overwrite it (will include user/award etc)
overwrite_existing = False

# reverse lookup dictionary for schema names
rev_schema_name = {}
for key, name in schema_name.items():
    rev_schema_name[name] = schema_name[key]

my_types = [i for i in ORDER if i in store.keys()]

second_round_items = {}

# Round I - only put the required - skip if exists already
for a_type in my_types:
    print(a_type)
    obj_type = rev_schema_name[a_type]
    # find required field
    schema_info = ff_utils.get_metadata('/profiles/{}.json'.format(a_type), key=transfer_key)
    req_fields = schema_info['required']
    ids = schema_info['identifyingProperties']
    first_fields = list(set(req_fields+ids))
    remove_existing_items = []
    counter=0
    print(len(store[a_type]), 'items exist on source')
    for an_item in store[a_type]:
        counter += 1
        
        if overwrite_existing:
            post_first = {key:value for (key,value) in an_item.items() if key in first_fields}
            ff_utils.post_metadata(post_first, obj_type, key = transfer_key)
        else:
            # does the item exist
            exists = False
            try:
                # TODO check with all identifiers
                existing = ff_utils.get_metadata(an_item['uuid'], key=transfer_key)
                exists = True
            except:
                exists = False
            # skip the items that exists
            if exists and existing:
                remove_existing_items.append(an_item['uuid'])
                print("{} {} can not post item".format(obj_type, an_item['uuid']))
                continue
            post_first = {key:value for (key,value) in an_item.items() if key in first_fields}
            ff_utils.post_metadata(post_first, obj_type, key = transfer_key)
   
    second_round_items[a_type] = [i for i in store[a_type] if i['uuid'] not in remove_existing_items]
    print(len(second_round_items[a_type]), 'items transfered to target')
    print()
    

user
9 items exist on source
user e2324f87-0625-4bbc-803b-d47677aebe08 can not post item
user 986b362f-4eb6-4a9c-8173-3ab267307e3a can not post item
user fb287a31-e765-41c5-8c1d-665f8e9f025b can not post item
user 83b5073a-069b-4162-9b30-6f42d5551e34 can not post item
user 986b362f-4eb6-4a9c-8173-3ab267228139 can not post item
user 986b362f-4eb6-4a9c-8173-3ab267111888 can not post item
user 68b09572-0c3e-48c7-90a9-7bad21d6bac1 can not post item
user d8ac229e-ec08-4411-be38-dc779520ea62 can not post item
1 items transfered to target

award
5 items exist on source
award 12a92962-8265-4fc0-b2f8-cf14f05db58b can not post item
award ae6c618f-7a8c-441e-a886-e30bbbe591da can not post item
award b0b9c607-f8b4-4f02-93f4-9895b461334b can not post item
award 43fc2984-a074-49c4-a69f-c9d3f3eadeba can not post item
award 029a2578-43dc-4343-8f41-694518cce304 can not post item
0 items transfered to target

lab
6 items exist on source
lab 828cd4fe-ebb0-4b36-a94a-d2e3a36cc989 can not post item
lab 3c577

In [11]:
# Round II - patch the rest of the metadata
for a_type in my_types:
    obj_type = rev_schema_name[a_type]
    if not second_round_items[a_type]:
        continue 
    for an_item in second_round_items[a_type]:
        counter += 1
        if a_type == 'file_fastq':
            if 'extra_files' in an_item:
                del an_item['extra_files']
        ff_utils.patch_metadata(an_item, obj_id = an_item['uuid'], key = transfer_key)

KeyboardInterrupt: 

In [23]:


for a_file in store['file_fastq']:
    acc = a_file['accession']    
    print(acc)
    clean_for_reupload(acc, transfer_key)
 

4DNFI1H51JMP
4DNFIGM134YI
4DNFIT8175E7
4DNFIV1I13LC
4DNFIC61767S
4DNFIB9O4GCD
4DNFIGML4U1D
4DNFIGBX67L2
4DNFIHGTSB82
4DNFIPN3FET8
4DNFI379J2SU
4DNFIMNQ96ER
4DNFIJPLOU2M
4DNFIG9352IW
4DNFINS7T3M5
4DNFI91KRBOG
4DNFIAPSBZH4
4DNFIV83OTT1
4DNFIJZQR7QV
4DNFICZJCKVK
4DNFITIUF4AC
4DNFIBGKIR8N
4DNFI1546BNF
4DNFIIBFU2LW
4DNFIZ6GWPN8
4DNFIJMR5I78
4DNFIIF2D1N1
4DNFIMOIGIGT
4DNFI3LES5HG
4DNFIC57YJDZ
4DNFIPCQFM46
4DNFI51VX8H4


In [ ]:
# Round III - move attachments
import boto3
s3 = boto3.resource('s3')

#source_addresses
source_health = ff_utils.get_metadata('/health', key = my_key)
source_raw = source_health['file_upload_bucket'] 
source_pf = source_health['processed_file_bucket'] 
source_att = source_health['blob_bucket']

#target_addresses
target_health = ff_utils.get_metadata('/health', key = transfer_key)
target_raw = target_health['file_upload_bucket'] 
target_pf = target_health['processed_file_bucket'] 
target_att = target_health['blob_bucket'] 

# Round III - move attachments
for a_type in my_types:
    obj_type = rev_schema_name[a_type]
    for an_item in second_round_items[a_type]:
        if 'attachment' in an_item.keys():
            at_key = an_item['attachment']['blob_id']
            copy_source = {'Bucket': source_att, 'Key': at_key}
            try:
                s3.meta.client.copy(copy_source, target_att, at_key)
            except:
                print('Can not find attachment on source', an_item['uuid'])
                continue
            print('attachment copied')


In [ ]:
# Round IV - move files
for a_type in my_types:
    if a_type in ['file_processed']:
        source_file_bucket = source_pf
        target_file_bucket = target_pf
    elif a_type in ['file_reference', 'file_fastq', 'file_microscopy', 'file_fasta', 'file_calibration']:
        source_file_bucket = source_raw
        target_file_bucket = target_raw
    else:
        continue
        
    for an_item in second_round_items[a_type]:
        # accumulate all keys from a file object to be uploaded
        files_to_upload = []
        file_resp = ff_utils.get_metadata(an_item['uuid'], key = my_key)
        # add extra file keys
        if file_resp.get('extra_files', []):
            for an_extra_file in file_resp['extra_files']:
                files_to_upload.append(an_extra_file['upload_key'])
        # add main file key
        files_to_upload.append(file_resp['upload_key'])
        
        for file_key in files_to_upload:
            copy_source = {'Bucket': source_file_bucket, 'Key': file_key}
            try:
                s3.meta.client.copy(copy_source, target_file_bucket, file_key)
            except:
                print('Can not find file on source', file_key)
                continue
            print('file copied')
            
